In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Dirs - must be absolute paths!
LOG_DIR = '/tmp/tf/mnist_cnn/'
MNIST_DIR = "/home/tkornuta/data/mnist"

# Set learning parameters.
LEARNING_RATE = 1e-4
BATCH_SIZE = 100
N_EPOCHS = 10


### A. Import MNIST datset, use one-hot encoding for labels.

In [2]:
mnist_dataset = input_data.read_data_sets(MNIST_DIR, one_hot=True)

Extracting /home/tkornuta/data/mnist/train-images-idx3-ubyte.gz
Extracting /home/tkornuta/data/mnist/train-labels-idx1-ubyte.gz
Extracting /home/tkornuta/data/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/tkornuta/data/mnist/t10k-labels-idx1-ubyte.gz


#### Helper functions

In [3]:
def fully_connected(inputs, input_height, input_width, input_depth, output_size, name):
    input_size = input_height*input_width*input_depth
    with tf.name_scope(name):
        w = tf.Variable(tf.random_normal(shape=[input_size,output_size], stddev=0.01), name="weights") 
        b = tf.Variable(tf.zeros(shape=[1,output_size]), name="bias")
        logits = tf.add(tf.matmul(inputs, w), b, name="logits")
        # Add histograms to TensorBoard.
        w_hist = tf.summary.histogram("w", w)
        b_hist = tf.summary.histogram("b", b)
        with tf.name_scope("Visualization"):
            # Interate through neurons.
            for output_neuron in range(output_size):
                w_activ = tf.slice(w, [0,output_neuron], [input_size,1])
                w_act_reshaped = tf.reshape(w_activ, [1,input_height,input_width,1])
                # Get activations from consecutive cols.
                image_summ = tf.summary.image("w_activations", w_act_reshaped)
        return logits

In [4]:
def fc_relu(inputs, input_width, input_height, input_depth, output_size, name):
    logits = fully_connected(inputs, input_height, input_width, input_depth, output_size, name)
    output = tf.nn.relu(logits)
    return output

In [5]:
def conv_relu(inputs, filters, kernel_size, name):
    with tf.name_scope(name):
        # Create a convolution layer
        conv = tf.layers.conv2d(
            inputs=inputs,
            filters=filters,
            kernel_size=kernel_size,
            padding="same",
            activation=tf.nn.relu)
        return conv

In [6]:
def pool(inputs, pool_size, strides, name):
    with tf.name_scope(name):
        pool = tf.layers.max_pooling2d(inputs=inputs, pool_size=pool_size, strides=strides)
        return pool

### B. Define the tensor graph.

In [10]:
# Reset graph - just in case.
tf.reset_default_graph()

# B. 
# 0. Placeholders for inputs.
with tf.name_scope("Input_data"):
    # Shape - none, as we will feed both training batches as well as test datasets.
    x = tf.placeholder(tf.float32, shape=None, name="x")
    targets = tf.placeholder(tf.float32, shape=None, name="target")
    with tf.name_scope("Visualization") as scope:
        x_image = tf.reshape(x, [-1,28,28,1])
        image_summ = tf.summary.image("Example_images", x_image)

# 1. Inference ops.
with tf.name_scope("Inference"):
    # Reshape inputs to batch to 2d, single-channel images.
    input_layer = tf.reshape(x, [-1, 28, 28, 1]) 
    
    # Conv + pooling 1
    conv1 = tf.layers.conv2d(input_layer, 32, [5, 5], activation=tf.nn.relu, padding="same", name="Conv1")
    pool1 = pool(conv1, pool_size=[2, 2], strides=2, name="Pool1")
    
    # Conv + pooling 2
    conv2 = tf.layers.conv2d(pool1, 64, [5, 5], activation=tf.nn.relu, padding="same", name="Conv2")
    pool2 = pool(conv2, pool_size=[2, 2], strides=2, name="Pool2")
    
    # Reshape output - flatten.
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    
    # FC layer + dropout
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu, name="FC1")
    #dense = fc_relu(inputs=pool2_flat, input_width, input_height, output_size, name):
    #dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dense, units=10, name="logits")
    #with tf.name_scope("fc"):
    #    w = tf.Variable(tf.random_normal(shape=[7*7*64,10], stddev=0.01), name="weights") 
    #    b = tf.Variable(tf.zeros(shape=[1,10]), name="bias")
    #    logits = tf.add(tf.matmul(pool2_flat, w), b, name="logits")

print ([v.name for v in tf.global_variables()])
with tf.variable_scope('Conv1') as scope:
    tf.get_variable_scope().reuse_variables()
    weights = tf.get_variable('kernel')
    print (weights)
    weights_summary = tf.summary.histogram("conv1_weights", weights)
    for f in range(weights.shape[3]):
        conv_filter = weights[:,:,0,f]
        conv_reshaped = tf.reshape(conv_filter, [1,5,5,1])
        # Get activations from consecutive cols.
        filter_summ = tf.summary.image("conv1_f"+str(f), conv_reshaped)
    
#tf.variable_scope('conv1', reuse=True)

# 2. Loss ops.
with tf.name_scope("Loss"):
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=targets)
    # Loss = mean over examples in the batch.
    loss = tf.reduce_mean(entropy)
    # Add loss summary.
    loss_summary = tf.summary.scalar("loss", loss)
    
# 3. Training ops.  
with tf.name_scope("Training"):
    optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(loss)
with tf.name_scope("Evaluation") as scope:
    # Count correct predictions by a simple argmax trick on each sample in a batch.
    correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(targets,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    # Add accuracy summary.
    accuracy_summary = tf.summary.scalar("accuracy", accuracy)

# Merge all summaries.
merged_sumaries = tf.summary.merge_all()

# 4. Init global variable.
init = tf.global_variables_initializer()

['Conv1/kernel:0', 'Conv1/bias:0', 'Conv2/kernel:0', 'Conv2/bias:0', 'FC1/kernel:0', 'FC1/bias:0', 'logits/kernel:0', 'logits/bias:0']
<tf.Variable 'Conv1/kernel:0' shape=(5, 5, 1, 32) dtype=float32_ref>


#### Helper functions

In [11]:
def feed_dict(dataset):
  """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
  if dataset=="train":
    xs, ys = mnist_dataset.train.next_batch(BATCH_SIZE)
  elif dataset=="valid":
    xs, ys = mnist_dataset.validation.images, mnist_dataset.validation.labels
  else: # test
    xs, ys = mnist_dataset.test.images, mnist_dataset.test.labels
  return {x: xs, targets: ys}

### C. Run session.

In [12]:
# Eventually clear the log dir.
if tf.gfile.Exists(LOG_DIR):
  tf.gfile.DeleteRecursively(LOG_DIR)
# Create (new) log dir.
tf.gfile.MakeDirs(LOG_DIR)

In [ ]:
# Create session.
sess = tf.InteractiveSession()
# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
train_writer = tf.summary.FileWriter(LOG_DIR + '/train', sess.graph)
valid_writer = tf.summary.FileWriter(LOG_DIR + '/valid')
test_writer = tf.summary.FileWriter(LOG_DIR + '/test')

# Initialize variables.
#tf.global_variables_initializer().run()
sess.run(init)

# First, check accuracy on test dataset.
#summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict("test"))
#test_writer.add_summary(summary, 0)
#print('Initial accuracy on test set: %s' % (acc))

# Perform training with minibatches and validation every 100 training steps.
n_batches = int(mnist_dataset.train.num_examples/BATCH_SIZE)
for e in range (N_EPOCHS):
  for b in range (n_batches):
    if b % 100 == 0:  # Record summaries and valid-set accuracy
      summary, acc = sess.run([merged_sumaries, accuracy], feed_dict=feed_dict("valid"))
      valid_writer.add_summary(summary, b+e*n_batches)
      print('Accuracy at step %s: %s' % (b+e*n_batches, acc))
    else:  # Record train set summaries, and train
      summary, acc = sess.run([merged_sumaries, optimizer], feed_dict=feed_dict("train"))
      train_writer.add_summary(summary, b+e*n_batches)

# Finally, check accuray on test dataset
summary, acc = sess.run([merged_sumaries, accuracy], feed_dict=feed_dict("test"))
test_writer.add_summary(summary, n_batches*N_EPOCHS)
print('Final accuracy on test set: %s' % (acc))

# Close writers and session.
train_writer.flush()
train_writer.close()
valid_writer.flush()
valid_writer.close()
test_writer.flush()
test_writer.close()
sess.close()

Accuracy at step 0: 0.0514
Accuracy at step 100: 0.8974
Accuracy at step 200: 0.9238
